### Гаусовы эллипсы

Самый простой способ детектирования аномалий - описать данные Гаусовым распределением и найти точки из его хвостов.

Пусть данные имеют Гауссово распределение с неизвестными нам вектором средних $\mu$ и матрицей ковариаций $\Sigma$
<img src="img/gaussian1.png" width=500>

Шаги:

1. Делаем эмпирическую оценку параметров $\mu$ и $\Sigma$:

    (методом максимальногоо правдоподобия)


2. Рассчитываем Махалонобисово расстояние до всех точек 

    $d_{(\mu,\Sigma)}(x_i)^2 = (x_i - \mu)'\Sigma^{-1}(x_i - \mu)$
    
    \* Махаланобисово расстояние - это метрика расстояния, учитывающая ковариации между признаками.


3. Отсекаем все что больше пороговой квантили (например в 97,5)% - это и есть выбросы

Множество точек с махаланобисовым расстоянием равным пороговому значению - эллипс, отсюда название метода. Всё что за пределами границы считаем выбросами, все что внутри - нормальными точками (inliers).

### Что не так с этим подходом? 

1) Во-первых, редко когда данные можно описать одним гауссовым эллипсом. В этом случае луше использовать GMM (он описываемый далее в этом докуенмте).

2) Во-вторых, выбросы сильно влияют на оценку $\mu$  и $\Sigma$. На картинке ниже сравнение двух эллипсов (эмпирического - синий и реального - красный). 

Видим, что синий эллипс сильно "притянут" выбросами. В результате пропускается очень много аутлайеров.

<img src="img/elliptic_empiric.png" width=400>

### Робастные эллипсы

В математике **робастной** называют оценку какого-либо параметра, не подверженную влиянию случайных выбросов.

Существует более робастная оценка матрицы ковариаций: MinCovDet (Min Covariance Determinant).


Принцип построения: 
1. выбираем параметр h
2. ищем подмножество точек объемом h, на котором определитель матрицы ковариаций минмален
3. ищем на этом подмножестве оценки параметров $\mu$ и $\Sigma$ методом маскимального правдоподобия - это и есть наши робастные оценки

Ниже сравнение ситуации с обычной оценкой ковариации (слева) и робастной (справа). По оси X отложены точки выборки, по оси Y - их махалонобисовы расстояния. Красная линия  - 97.5-процентный уровень эллипса.
<img src="img/elliptic_comparison.png" width=600>

Видим, что при использовании робастной оценки, махаланобисовы расстояния выбросов стали больше => выбросы лучше отделяются.

Для реализации нужен полный перебор всех подмножеств: $\binom{n}{h}$ => считается долго. Но в 1999 году появился feasible вариант алгоритма (fastMCD) и его стали использовать.

### Sklearn :: EllipseEnvelope

В sklearn обычная оценка матрицы ковариаций считается через covariance.EmpiricalCovariance.

In [ ]:
from sklearn.covariance import EmpiricalCovariance
cov = EmpiricalCovariance().fit(X)

Робастная оценка считается covariance.MinCovDet.

In [ ]:
from sklearn.covariance import MinCovDet
mcd = MinCovDet().fit(X)

В этих же классах есть метод для расчета Махаланобисова расстояния при раасчитанных ($\mu$,$\Sigma$)

In [ ]:
cov.mahalanobis(X_test)
mcd.mahalanobis(X_test)

In [ ]:
cov.error_norm(another_cov) - норма разницы с другой матрицей ковариаций
(норма Фробениуса - сумма квадратов элементов)

### Gaussian Mixture Model

Вариация на тему предыдущего подхода - когда данные моделируются комбинацией сразу нескольких гауссовых рапсределений. Такая конфигурация называется GMM (gaussian mixture model).

<img src="img/gmm.png" width=400>

Она позволяет уйти от требуемой симметричности данных и моделировать более сложные выборки.

In [ ]:
from sklearn.mixture import GasussianMixture
gmm = GaussianMixture(n_components = 3, covariance_type='full')
gmm.fit(X)

Чтобы оценить удаленность точки от центра своего распределения, можем посчитать вер-ть методом score_samples()

In [ ]:
point_dist = gmm.score_samples(X_test)

### Local Outlier Factor

Алгоритм Относится к классу плотностных (density based) методов. 

**Идея**: если "плотность" выборки в районе точки небольшая, то скорее всего это выброс. 

Что значит "плотность"?
    
    Плотность считается как 1 / среднее расстояние до k-ближайших соседей. 

Что значит "небольшая"? 

    Значит, что плотность точки сильно меньше, чем плотность её соседей. 
    
Показатель LOF = отношение средней плотности соседей к плотности точки. Пример значений LOF для разных точек ниже.

<img src="img/lof.png" width=400>



Небольшой нюанс в том, что в расстояния считается не Евклидово, а Reachibility Distance - оно дополнительно ограничивается снизу. Иллюстраиця ниже - расстояние от C до A равно расстоянию от B до A, а вот расстояние от D до A больше.
<img src="img/lof_dist.png" width=150>

Сделано (как я понял) для устойчивости расчета LOF (чтобы много маленьких расстояний не влияли на среднее значение).

Можно заметить, что в этом методе исп тот же концепт, что и в плотностном алгоритме кластеризации DBSCAN.

Объяснение для чайников:
https://blog.stealthbits.com/local-outlier-factor-part-1

### Sklearn :: LocalOutlierFactor

В Sklearn для LOF есть класс LocalOutlierFactor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=20)
y_pred = clf.fit_predict(X)

В Sklearn все методы детектирования аномалий имеют очень схожий API:

Методы:
- fit - считается "нестандартность" каждой точки (у каждого алгоритма метрика соотвественно своя)
- predict - с помощью порогового значения каждая точка помечаются как выброс или нет

Метод predict() возвращает значение :
- 1, если точка не считается выбросом
- -1, если точка прогнозируется выбросом

Параметры обучения:
- contamination_factor - верхний квантиль расстояния, в котором точка считается выбросом (обычно 10%)



Это единственный anomaly detection метод в sklearn, который можно распараллелить

In [ ]:
clf = LocalOutlierFactor(n_neighbors=20, n_jobs=16)
y_pred = clf.fit_predict(X)